In [38]:
import os
import sys


import pytest
import pandas as pd
from sklearn.model_selection import train_test_split


from prediction_delivery_time.processing.data_manager import load_dataset
from prediction_delivery_time import __version__ as _version
from prediction_delivery_time.config.core import config

path = "/Users/waresix/Documents/New Project/cache/prediction_delivery_time/prediction_delivery_time/tests"

PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.dirname(path), 
                  os.pardir))

sys.path.append(PROJECT_ROOT)

def dataset():
    data = load_dataset(file_name=config.app_config.data)

    return data

def data_train_test():
    data = load_dataset(file_name=config.app_config.data)

    X_train, X_test, y_train, y_test = train_test_split(
        data[config.config_model.features],  # predictors
        data[config.config_model.target],
        test_size=config.config_model.test_size,
        # we are setting the random seed here
        # for reproducibility
        random_state=config.config_model.random_state,
    )

    return X_train, X_test, y_train, y_test


def data_sample():
    data = load_dataset(file_name=config.app_config.data)

    X_train, _, _, _ = train_test_split(
        data[config.config_model.features],  # predictors
        data[config.config_model.target],
        test_size=config.config_model.test_size,
        # we are setting the random seed here
        # for reproducibility
        random_state=config.config_model.random_state,
    )
    
    data_sample = X_train.iloc[:1]

    return data_sample


def dummy_df():
    dummy_data = {
        "ID": [
            "4607"
        ],
        "Delivery_person_ID": [
            "INDORES13DEL02"
        ],
        "Delivery_person_Age": [
            37
        ],
        "Delivery_person_Ratings": [
            4.9
        ],
        "Restaurant_latitude": [
            22.745049
        ],
        "Restaurant_longitude": [
            75.892471
        ],
        "Delivery_location_latitude": [
            22.765049
        ],
        "Delivery_location_longitude": [
            75.912471
        ],
        "Type_of_order": [
            "Snack"
        ],
        "Type_of_vehicle": [
            "motorcycle"
        ],
        "Time_taken": [
            24
        ]
    }
    df_dummy_data = pd.DataFrame(dummy_data)
    return df_dummy_data

drop features

In [50]:
from prediction_delivery_time import pipeline

data_sample = data_train_test()

 # act
result_trim_whitespaces = pipeline.price_pipe[:1].fit_transform(data_sample) 
result_trim_whitespaces_col = list(result_trim_whitespaces)

# assert
#'Restaurant_latitude' not in result_trim_whitespaces_col #'Type_of_order
('Type_of_order' not in result_trim_whitespaces_col) == False

True

test_trim_whitespaces

In [86]:
result_trim_whitespaces = pipeline.price_pipe[:2].fit_transform(data_sample) 
result_trim_whitespaces = result_trim_whitespaces['Type_of_order'].values[0]

result_trim_whitespaces

'Buffet'

In [51]:
data_sample_type_of_order = data_sample.iloc[0].Type_of_order
data_sample_type_of_order[-1] == ' '

True

distance column

In [71]:
 # act
result_trim_distance_column = pipeline.price_pipe[:3].fit_transform(data_sample) 

result_trim_distance_column['distance'].values[0] == pytest.approx(6.2, 0.1)

True

type vehicle map

In [75]:
 # act
result_trim_vehicle_map = pipeline.price_pipe[:4].fit_transform(data_sample) 

result_trim_vehicle_map['Type_of_vehicle'].dtype == 'int64'
result_trim_vehicle_map['Type_of_vehicle'].values[0] == 0

True

type order map

In [81]:
 # act
result_trim_type_order_map = pipeline.price_pipe[:5].fit_transform(data_sample) 

result_trim_type_order_map['Type_of_order'].dtype == 'int64'
result_trim_type_order_map['Type_of_order'].values[0] == 2

True

Model

### Test_train_pipeline

In [97]:
from prediction_delivery_time import train_pipeline

X_train, X_test, y_train, y_test = data_train_test()

data_sample = data_sample()

pipeline.price_pipe.fit(X_train, y_train) 

result = pipeline.price_pipe.predict(data_sample)

In [107]:
from prediction_delivery_time import train_pipeline

a = train_pipeline.run_training()
a is None

True

### Test predict

In [12]:
from prediction_delivery_time.predict import make_prediction

data_dummy = dummy_df()

a = make_prediction(input_data=data_dummy)

a#['errors'] is None
a['predictions'][0][0]

23.906107

### Test processing

data manager

In [18]:
# load_pipeline
from prediction_delivery_time.config.core import config
from prediction_delivery_time.processing.data_manager import load_pipeline, load_dataset

pipeline_file_name = f"{config.app_config.pipeline_save_file}{_version}.pkl"
_price_pipe = load_pipeline(file_name=pipeline_file_name)

_price_pipe is not None

/Users/waresix/Library/Caches/pypoetry/virtualenvs/prediction-delivery-time-GBb6yWNt-py3.9/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.3.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


True

In [24]:
# load dataset
a = load_dataset(file_name=config.app_config.data)
a.shape == (45593, 11)

45593

features

In [34]:
from prediction_delivery_time.processing.features import TrimWhitespaces

b = TrimWhitespaces(
                variables=config.config_model.trim_whitespace_vars
            )
a = b.fit_transform(X=data_dummy, y=[0])
a.Type_of_order.values[0][-1]

'k'

validation

In [41]:
from prediction_delivery_time.processing.validation import validate_inputs

data = dataset()

validated_data, errors = validate_inputs(input_data=data)

validated_data.shape

(45593, 11)

In [60]:
from prediction_delivery_time.processing.validation import validate_inputs

data = dataset()

data.loc[0, "Type_of_order"] = 1

validated_data, errors = validate_inputs(input_data=data)

assert errors

'[{"type":"string_type","loc":["inputs",0,"Type_of_order"],"msg":"Input should be a valid string","input":1,"url":"https://errors.pydantic.dev/2.6/v/string_type"}]'

### Config

In [65]:
from prediction_delivery_time.config.core import create_and_validate_config

a = create_and_validate_config()
a.config_model.test_size

0.1

In [20]:
from fastapi.testclient import TestClient
from serve.app.app import app

client = TestClient(app)

response = client.get("/")
response
response.json()

{'name': 'prediction_delivery_time_model',
 'description': '',
 'version': '0.0.1'}

In [16]:
response = client.get("/test")
response

404

In [9]:
response = client.post("/predict")
response

<Response [422 Unprocessable Entity]>

In [19]:
response = client.get("/monitoring-models")
response.is_error

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/waresix/Library/Caches/pypoetry/virtualenvs/prediction-delivery-time-GBb6yWNt-py3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/5w/bjfc8rgd6b1b9hw70b61ry1c0000gn/T/ipykernel_51143/2550967393.py", line 1, in <module>
    response = client.get("/monitoring-models")
  File "/Users/waresix/Library/Caches/pypoetry/virtualenvs/prediction-delivery-time-GBb6yWNt-py3.9/lib/python3.9/site-packages/starlette/testclient.py", line 502, in get
    return super().get(
  File "/Users/waresix/Library/Caches/pypoetry/virtualenvs/prediction-delivery-time-GBb6yWNt-py3.9/lib/python3.9/site-packages/httpx/_client.py", line 1055, in get
    return self.request(
  File "/Users/waresix/Library/Caches/pypoetry/virtualenvs/prediction-delivery-time-GBb6yWNt-py3.9/lib/python3.9/site-packages/starlette/testclient.py", line 468, in request
    return s

In [6]:
a = {
  "inputs": [
    {
      "Delivery_location_latitude": 22.765049,
      "Delivery_location_longitude": 75.912471,
      "Delivery_person_Age": 37,
      "Delivery_person_ID": "INDORES13DEL02",
      "Delivery_person_Ratings": 4.9,
      "ID": "4607",
      "Restaurant_latitude": 22.745049,
      "Restaurant_longitude": 75.892471,
      "Time_taken": 24,
      "Type_of_order": "Snack",
      "Type_of_vehicle": "motorcycle"
    }
  ]
}

response = client.post("/predict", json=a)
response = response.json()
response['predictions'][0][0]

2024-02-19 19:06:52,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-19 19:06:52,076 INFO sqlalchemy.engine.Engine INSERT INTO prediction ("ID", "Delivery_person_ID", "Delivery_person_Age", "Delivery_person_Ratings", "Restaurant_latitude", "Restaurant_longitude", "Delivery_location_latitude", "Delivery_location_longitude", "Type_of_order", "Type_of_vehicle", "Time_taken", prediction, "createdAt", "updatedAt") VALUES (%(ID)s, %(Delivery_person_ID)s, %(Delivery_person_Age)s, %(Delivery_person_Ratings)s, %(Restaurant_latitude)s, %(Restaurant_longitude)s, %(Delivery_location_latitude)s, %(Delivery_location_longitude)s, %(Type_of_order)s, %(Type_of_vehicle)s, %(Time_taken)s, %(prediction)s, now(), now()) RETURNING prediction."createdAt", prediction."updatedAt"
2024-02-19 19:06:52,077 INFO sqlalchemy.engine.Engine [generated in 0.00065s] {'ID': '4607', 'Delivery_person_ID': 'INDORES13DEL02', 'Delivery_person_Age': 37, 'Delivery_person_Ratings': 4.9, 'Restaurant_latitude': 22.745049

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "prediction_pkey"
DETAIL:  Key ("ID")=(4607) already exists.

[SQL: INSERT INTO prediction ("ID", "Delivery_person_ID", "Delivery_person_Age", "Delivery_person_Ratings", "Restaurant_latitude", "Restaurant_longitude", "Delivery_location_latitude", "Delivery_location_longitude", "Type_of_order", "Type_of_vehicle", "Time_taken", prediction, "createdAt", "updatedAt") VALUES (%(ID)s, %(Delivery_person_ID)s, %(Delivery_person_Age)s, %(Delivery_person_Ratings)s, %(Restaurant_latitude)s, %(Restaurant_longitude)s, %(Delivery_location_latitude)s, %(Delivery_location_longitude)s, %(Type_of_order)s, %(Type_of_vehicle)s, %(Time_taken)s, %(prediction)s, now(), now()) RETURNING prediction."createdAt", prediction."updatedAt"]
[parameters: {'ID': '4607', 'Delivery_person_ID': 'INDORES13DEL02', 'Delivery_person_Age': 37, 'Delivery_person_Ratings': 4.9, 'Restaurant_latitude': 22.745049, 'Restaurant_longitude'